# TESTING HYPERPARAMETERS
# TRANSFORMER. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff. 
### model = Transformer(num_layers=2, d_model=72, num_heads=1)

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2023-12-05 14:37:56.488268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 14:37:56.488361: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 14:37:56.491009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 14:37:56.742472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 14:37:59.168289: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_train/")
ds_val = tf.data.Dataset.load("../data/ds_val/")
ds_test = tf.data.Dataset.load("../data/ds_test/")

2023-12-05 14:38:03.192696: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 14:38:03.472371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 14:38:03.472451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 14:38:03.476747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-05 14:38:03.476830: I tensorflow/compile

In [3]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [4]:
def transform_element(features, context, labels):
    # transform lebels from (1,) to (2,)
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return (features, context), labels[0]

In [5]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [6]:
for x, y in ds_train.batch(1024).take(1):
    print(x[0].shape)
    print(x[1].shape)
    print(y.shape)

(1024, 64, 8)
(1024, 4)
(1024, 2)


In [7]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED, reshuffle_each_iteration=False).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.lstm = tf.keras.layers.LSTM(d_model // 8 * 6, return_sequences=True)
        self.embed_dd = tf.keras.layers.Embedding(100, d_model // 8)
        self.embed_plate = tf.keras.layers.Embedding(100, d_model // 8)
        self.conc = tf.keras.layers.Concatenate()
        self.pos_encoding = tf.keras.layers.Embedding(100, d_model)

    def call(self, x):
        cont, plate, dd = x[:,:,:-2], x[:,:,-2], x[:,:,-1] 
        x1 = self.lstm(cont)
        x2 = self.embed_dd(dd)
        x3 = self.embed_plate(plate)
        x = self.conc([x1, x2, x3])
        x_pos = self.pos_encoding(tf.range(x1.shape[1]))
        x = x + x_pos
        return x

In [9]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [10]:
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [11]:
class CrossAttention(BaseAttention):
	def call(self, x, context):
		attn_output = self.mha(
			query=x,
			key=context,
			value=context)
		x = self.add([x, attn_output])
		x = self.layernorm(x)
		return x

In [12]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [13]:
class EncoderBlock(tf.keras.layers.Layer):
	def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
		super().__init__()

		self.self_attention = GlobalSelfAttention(
			num_heads=num_heads,
			key_dim=d_model,
			dropout=dropout_rate)

		self.ffn = FeedForward(d_model, dff)

	def call(self, x):
		x = self.self_attention(x)
		x = self.ffn(x)
		return x

In [14]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads,
               dff, dropout_rate=0.1):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.dense = tf.keras.layers.Dense(d_model)
        self.embed_plate = tf.keras.layers.Embedding(100, d_model)
        self.conc = tf.keras.layers.Concatenate(axis=1)
        self.enc_blocks = [
            EncoderBlock(d_model=d_model,
                        num_heads=num_heads,
                        dff=dff,
                        dropout_rate=dropout_rate)
            for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x):
        x = self.dropout(x)
        x = tf.reshape(x, (-1, 4, 1))
        cont, plate = x[:, :-1, :], x[:, -1, :]
        x1 = self.dense(cont)
        x2 = self.embed_plate(plate)
        x = self.conc([x1, x2])
        for block in self.enc_blocks:
            x = block(x)

        return x  # Shape `(batch_size, seq_len, d_model)`.

In [15]:
class DecoderBlock(tf.keras.layers.Layer):
    def __init__(self,
                d_model,
                num_heads,
                dff,
                dropout_rate=0.1):
        super(DecoderBlock, self).__init__()

        self.causal_self_attention = GlobalSelfAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.cross_attention = CrossAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=dropout_rate)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x, context):
        x = self.causal_self_attention(x=x)
        x = self.cross_attention(x=x, context=context)
        x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
        return x

In [16]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, dropout_rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.dec_blocks = [
            DecoderBlock(d_model=d_model, num_heads=num_heads,
                        dff=dff, dropout_rate=dropout_rate)
            for _ in range(num_layers)]

    def call(self, x, context):
        x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
        x = self.dropout(x)
        for block in self.dec_blocks:
            x = block(x, context)
        # The shape of x is (batch_size, target_seq_len, d_model).
        return x

In [17]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads,
                dropout_rate=0.1):
        super().__init__()
        self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                            num_heads=num_heads, dff=4*d_model,
                            dropout_rate=dropout_rate)

        self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=4*d_model,
                           dropout_rate=dropout_rate)

        self.final_layer = tf.keras.layers.Dense(2)

    def call(self, inputs):
        x, context = inputs
        context = self.encoder(context)
        x = self.decoder(x, context) # (batch_size, target_len, d_model)
        x = x[:, -1, :]
        logits = self.final_layer(x)  # (batch_size, 1, target_vocab_size)
        return logits

In [18]:
model = Transformer(num_layers=2, d_model=32, num_heads=8)

In [19]:
model.build(input_shape=[(None, BLOCK_SIZE, 8), (None, 4)])
model.count_params()

245762

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0004),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [21]:
history = model.fit(ds_train, epochs=30, validation_data=ds_val)

Epoch 1/30


2023-12-05 14:38:29.193219: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 200 of 2000
2023-12-05 14:38:49.179922: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 578 of 2000
2023-12-05 14:38:59.211582: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 785 of 2000
2023-12-05 14:39:19.267608: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1060 of 2000
2023-12-05 14:39:39.185043: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1406 of 2000
2023-12-05 14:39:49.204786: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take 

3227/3227 [==============================] - 1165s 319ms/step - loss: 0.1706 - accuracy: 0.9439 - f1_score: 0.6355 - val_loss: 0.1885 - val_accuracy: 0.9424 - val_f1_score: 0.5754
Epoch 2/30


2023-12-05 14:57:44.410863: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 173 of 2000
2023-12-05 14:57:54.443001: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 339 of 2000
2023-12-05 14:58:14.425086: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 694 of 2000
2023-12-05 14:58:24.429605: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 857 of 2000
2023-12-05 14:58:44.421202: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1193 of 2000
2023-12-05 14:58:54.438649: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a

3227/3227 [==============================] - 1253s 351ms/step - loss: 0.1586 - accuracy: 0.9474 - f1_score: 0.6665 - val_loss: 0.1861 - val_accuracy: 0.9428 - val_f1_score: 0.5800
Epoch 3/30


...

3227/3227 [==============================] - 1117s 293ms/step - loss: 0.1257 - accuracy: 0.9573 - f1_score: 0.7637 - val_loss: 0.2044 - val_accuracy: 0.9410 - val_f1_score: 0.6714
Epoch 29/30


2023-12-06 02:05:47.683209: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 111 of 2000
2023-12-06 02:06:07.689964: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 337 of 2000
2023-12-06 02:06:27.681179: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 580 of 2000
2023-12-06 02:06:37.768429: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 674 of 2000
2023-12-06 02:06:47.768484: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 769 of 2000
2023-12-06 02:07:07.662806: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a 

3227/3227 [==============================] - 1340s 360ms/step - loss: 0.1242 - accuracy: 0.9578 - f1_score: 0.7675 - val_loss: 0.2204 - val_accuracy: 0.9364 - val_f1_score: 0.6371
Epoch 30/30


2023-12-06 02:28:07.889637: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 63 of 2000
2023-12-06 02:28:18.413760: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 124 of 2000
2023-12-06 02:28:37.887584: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 264 of 2000
2023-12-06 02:28:47.952279: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 326 of 2000
2023-12-06 02:29:08.063857: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 479 of 2000
2023-12-06 02:29:27.846346: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3227/3227 [==============================] - 1428s 358ms/step - loss: 0.1240 - accuracy: 0.9577 - f1_score: 0.7669 - val_loss: 0.1986 - val_accuracy: 0.9378 - val_f1_score: 0.6413


In [22]:
model.evaluate(ds_test)

100/100 [==============================] - 27s 263ms/step - loss: 0.0754 - accuracy: 0.9793 - f1_score: 0.6724


[0.07541481405496597,
 0.9793444275856018,
 array([0.9895041 , 0.35529557], dtype=float32)]

In [23]:
model.save("../models/model_v10.keras")

In [24]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v10.csv", index=False)